In [51]:
from nltk.tokenize.casual import casual_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import pandas as pd
import string
import os

input_filepath="../../data/clean/"
filename="chao_by_lga_and_from_user_name.csv"

In [35]:
tweetdf=pd.read_csv(os.path.join(input_filepath, filename),header=0, keep_default_na=False)
tweetdf

,Unnamed: 0,lga,from_user_name,text,cleaned
0,0,Bayside,0731Sus,うまそー！ #chocolatecafe#rockdale 場所: Sir Braxton ...,umaso #chocolatecafe #rockdale chang suo sir b...
1,1,Bayside,11Bass,Sydney roads! #gottoloveapun #sydneycitylife #...,sydney roads #gottoloveapun #sydneycitylife #t...
2,2,Bayside,17maez17,I’m gonna miss you so much my Simon😘😘😘 #lovelo...,i'm gonna miss you so much my simon #lovelove ...
3,3,Bayside,1JYK1,#flightattendantlife #InstructorLife @qantas #...,#flightattendantlife #instructorlife #a380 air...
4,4,Bayside,3DogsBums,"@smh my experience, about 5,000 people waiting...","my experience about 5,000 people waiting"
5,5,Bayside,4fthawaiian,Had our favorite #fishandchips for dinner ❤️ @...,had our favorite #fishandchips for dinner rams...
6,6,Bayside,545strat,Choc banana shake with choc almond milk...not ...,choc banana shake with choc almond milk ... no...
7,7,Bayside,577Express,暑い！！ 顔認証の自動入国審査できんかった。。 顔変わったかな(￣∇￣) #australi...,shu yan ren zheng nozi dong ru guo shen cha de...
8,8,Bayside,76921890,I'm at Sydney International Airport - Arrivals...,i'm at sydney international airport arrivals i...
9,9,Bayside,ACampbell47,Just a quick trip up to #Brisbane and back tod...,just quick trip up to #brisbane and back today...


In [69]:
# Set up Vectorizer for use with pre-cleaned and tokenized data
def smc_tokeniser(doc):
    return doc.split(' ')

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=smc_tokeniser,
    ngram_range=(1,3),
    max_features=5000,
    stop_words='english',
    token_pattern=None)  

In [70]:
docs=tweetdf['cleaned']

tfidf_vectors = tfidf.fit_transform(docs)
len(tfidf.vocabulary_)

features = np.array(tfidf.get_feature_names())
print('Number of features', len(features))


Number of features 5000


In [40]:
def top_tfidf_feats(row, features, top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

#Return top features amongst documents in rows indentified by indices in grp_ids
def top_mean_feats(tfs, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    if grp_ids:
        D = tfs[grp_ids].toarray()
    else:
        D = tfs.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

In [72]:
top_features = top_mean_feats(tfidf_vectors,features, top_n=500)

In [73]:
# Something has gone arry with the tokenizer since fullstop and empty string are
# appearing as top features - need to address this eventually
top_features
#top_features.iloc[47].feature in string.punctuation
#'' in string.punctuation

,feature,tfidf
0,...,0.061964
1,sydney,0.034169
2,nsw,0.019612
3,.,0.019286
4,#sydney,0.018199
5,i'm,0.017128
6,beach,0.017030
7,new,0.015399
8,day,0.013743
9,australia,0.013206


In [74]:
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=features)

In [76]:
tfidf_df.describe()

,,,,.,...,.,.,. .,. lswrh,...,...,🥂,🥃,🥊,🥑,🦀,🦄,🦋,🧀,🧜,🧡
count,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,...,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000,14431.000000
mean,0.003541,0.000996,0.000390,0.000444,0.000219,0.001158,0.000212,0.000555,0.000012,0.000351,...,0.001373,0.000272,0.000317,0.000295,0.000288,0.000557,0.000469,0.000180,0.000327,0.000704
std,0.044218,0.017257,0.012688,0.009666,0.006382,0.015382,0.008049,0.009458,0.000871,0.008452,...,0.021100,0.010773,0.014147,0.011040,0.009943,0.015842,0.014588,0.007539,0.010443,0.015073
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,0.569410,0.652475,0.345701,0.294196,0.398157,0.434181,0.285629,0.065599,0.458245,...,0.656769,0.690732,0.963149,0.771155,0.608180,0.885165,0.860931,0.573092,0.596392,0.551783


In [82]:
tfidf_df = tweetdf[['lga','from_user_name']].join(tfidf_df)

In [83]:
tfidf_df

,lga,from_user_name,,,,.,...,.,.,. .,...,🥂,🥃,🥊,🥑,🦀,🦄,🦋,🧀,🧜,🧡
0,Bayside,0731Sus,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,Bayside,11Bass,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Bayside,17maez17,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayside,1JYK1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Bayside,3DogsBums,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
5,Bayside,4fthawaiian,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6,Bayside,545strat,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
7,Bayside,577Express,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
8,Bayside,76921890,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
9,Bayside,ACampbell47,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [84]:
tfidf_df.to_csv('../../data/clean/chao_tfidf_by_lga_and_from_user_name.csv')